In [1]:
#  This link for dataset you can run & download it befor run other cells

# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
import torch
print(f"Is ROCm/CUDA available? {torch.cuda.is_available()}")
print(f"Device Name: {torch.cuda.get_device_name(0)}")

Is ROCm/CUDA available? True
Device Name: AMD Radeon Graphics


/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory
/opt/amdgpu/share/libdrm/amdgpu.ids: No such file or directory


In [3]:
with open("tiny_shakespeare.txt",'r',encoding='utf-8') as f:
    text=f.read()

print("length of the dataset in characters:",len(text))

length of the dataset in characters: 1115394


In [4]:
# characters nano_gpt knows
chars = sorted (list(set(text)))
# Vocabulary size
vocb_size = len(chars)
print("".join(chars))
print(f"length of vocabulary: {vocb_size}")


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
length of vocabulary: 65


In [5]:
# create a encoder and decoder
stoi={ch:i for i,ch in enumerate(chars)} # string to integer 
itos={i:ch for i,ch in enumerate(chars)} # integer to string

encode = lambda s:[stoi[c] for c in s] # take a string and output a list of integers
decode = lambda l:''.join([itos[i] for i in l]) # take a list of integers and output a string

print("testing encoder and decoder")
print(encode("hii there..."))
print(decode(encode("hii there...")))


testing encoder and decoder
[46, 47, 47, 1, 58, 46, 43, 56, 43, 8, 8, 8]
hii there...


In [6]:
data = torch.tensor(encode(text),dtype=torch.long)
print(f"data shape: {data.shape} and dtype: {data.dtype}")
print(data[0:1000])

data shape: torch.Size([1115394]) and dtype: torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1,

In [7]:
# let spit data into train and validation sets for training a tranformer
n = int(0.9*len(text))
train_data = data[:n] # first 90% for training 
val_data = data[n:]

block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [8]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when the input is this: {context} the target must be: {target}")

when the input is this: tensor([18]) the target must be: 47
when the input is this: tensor([18, 47]) the target must be: 56
when the input is this: tensor([18, 47, 56]) the target must be: 57
when the input is this: tensor([18, 47, 56, 57]) the target must be: 58
when the input is this: tensor([18, 47, 56, 57, 58]) the target must be: 1
when the input is this: tensor([18, 47, 56, 57, 58,  1]) the target must be: 15
when the input is this: tensor([18, 47, 56, 57, 58,  1, 15]) the target must be: 47
when the input is this: tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target must be: 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]])
    return x,y

xb ,yb = get_batch('train')
print('inputs: ')
print(xb.shape)
print(xb)

print('targets: ')
print(yb.shape)
print(yb)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b,:t+1]
        terget = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")